In [5]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [7]:
from sklearn import datasets
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import pandas as pd

df_news = pd.read_excel('CTG.xls', header=None) 
df_news
df = df_news.values

data = df[:,0:41]
label = df[:,-1]-1

data, label = shuffle(data, label, random_state = 3)
l = int(1/5*len(label))
testdata = data[: l]
testlabel = label[: l]
print(len(testdata[0]))
data = data[l: ]
label = label[l: ]
print(len(label))
n_class = len(unique(label))

41
1701


In [8]:
def calc_NN(X_train, X_test):
    k=X_train.shape[0]
    Diff= X_train - X_test
    A=reshape(Diff,[k,-1])
    S=sum(A**2,axis=1)
    order=argsort(S)
    
    return order

In [15]:
def CALC_NN(X_train, y_train): 
    indicesByDistance=[]
    labels_save = []
    for i in range(y_train.shape[0]):
        order=np.array(calc_NN(X_train, X_train[i]))
        indicesByDistance.append(order)
        labels=y_train[order]
        labels_save.append(labels)
        
    return labels_save, indicesByDistance

In [16]:
def find_sig(t):
    f=where(t==True)[0]
    if f.shape[0]==0:
        return np.NaN
    else:
        return f[0] + 1

In [17]:
def change_label(labels_save, test_index, A, R ,M):
    C = []
    L = labels_save[test_index]
    orilabel = labels_save[test_index][0]
    sig = []
    for i in range(2):
        C.append(np.cumsum(L==i))
    
    C = array(C)[:, 1:M+1]
        
    class_keep = C[orilabel] > A[1:M+1]
    class_change = C[orilabel] < R[1:M+1]
    
    acceptIdx = find_sig(class_keep) 
    rejectIdx = find_sig(class_change)
    
    if acceptIdx is np.NaN and rejectIdx is np.NaN:
        if abs(R[-1]-C[orilabel][-1]) < abs(A[-1]-C[orilabel][-1]):
            return 1-orilabel
        else:
            return orilabel
            
    elif acceptIdx is np.NaN:
        return 1-orilabel
    elif rejectIdx is np.NaN:
        return orilabel
    elif acceptIdx < rejectIdx:
        return orilabel
    else:
        return 1-orilabel
    
def change_label_multi(labels_save, test_index, A, R ,M):
    L = labels_save[test_index]
    orilabel = labels_save[test_index][0]
    sig = []

    C = array(cumsum(L[1:] == orilabel))[:M]
        
    class_keep = C > A[1:M+1]
    class_change = C < R[1:M+1]
    
    acceptIdx = find_sig(class_keep) 
    rejectIdx = find_sig(class_change)

    if acceptIdx is np.NaN and rejectIdx is np.NaN:
        if abs(R[-1]-C[-1]) < abs(A[-1]-C[-1]):
            return L[1]
        else:
            return orilabel
            
    elif acceptIdx is np.NaN:
        return Counter(L[1: rejectIdx+1]).most_common(1)[0][0]
    elif rejectIdx is np.NaN:
        return orilabel
    elif acceptIdx < rejectIdx:
        return orilabel
    else:
        return Counter(L[1: rejectIdx+1]).most_common(1)[0][0]




In [18]:
alpha = 0.05
Beta = arange(0.05, 1, 0.2)
P0 = arange(0.05, 1, 0.15)
P1 = arange(0.05, 1, 0.15)
M = 80
ks = 5

In [20]:
from sklearn.model_selection import KFold
from collections import Counter
acc = [0]
kf = KFold(n_splits=5)
hyperpara = []

labelss = []
labels_save = []

for beta in Beta:
    for p0 in P0:
        for p1 in P1:
            if p0 >= p1:
                continue

            acc_tmp = []
            m = arange(-1, M)+1
            denominator = log(p1/p0) - log((1-p1)/(1-p0))

            R = log(beta/(1-alpha)) / denominator + m *  log((1-p0)/(1-p1)) / denominator
            A = log((1-beta)/alpha) / denominator + m * log((1-p0)/(1-p1)) / denominator
            Count = 0
            
            for train_index, test_index in kf.split(data):
                res = []
                
                X_train, X_test = data[train_index], data[test_index]
                y_train, y_test = label[train_index], label[test_index]

                if len(labelss) < 5: 
                    labels_save, _ = CALC_NN(X_train, y_train)  
                    labelss.append(labels_save)
                else:
                    labels_save = labelss[Count]
                Count += 1
############### strategy 1 #############################
                for i in range(y_train.shape[0]):
                    if n_class > 2:
                        y_train[i] = change_label_multi(labels_save, i, A, R, M)
                    else:
                        y_train[i] = change_label(labels_save, i, A, R ,M)
############### strategy 1 #############################
                                
                for i in range(y_test.shape[0]):
                    order=np.array(calc_NN(X_train, X_test[i]))
                    labels=y_train[order]
                    res.append(Counter(labels[:ks]).most_common(1)[0][0])

                acc_tmp.append(sum(res == y_test)/y_test.shape[0]) 
            currAcc = array(acc_tmp).mean()
            
            sd = sqrt(sum([(x-currAcc)**2 for x in acc_tmp])/5) 
            
            acc.append(currAcc)
            if currAcc == max(acc):
                hyperpara = [alpha, beta, p0, p1]
            print("************************************************************")
            print("alpha:{}, beta:{}, p0:{}, p1:{}, k:{}".format(alpha,beta,p0,p1,ks))
            print("H0: p =", p0 ," ************ ", "H1: p =", p1)            
            print("Accuracy:{}".format(acc[-1]))
            print("standard deviation:{}".format(sd))
            print("The highest accuracy so far:", max(acc))
            
            

alpha = hyperpara[0]
beta = hyperpara[1]
p0 = hyperpara[2]
p1 = hyperpara[3]
denominator = log(p1/p0) - log((1-p1)/(1-p0))
R = log(beta/(1-alpha)) / denominator + m * log((1-p0)/(1-p1)) / denominator
A = log((1-beta)/alpha) / denominator + m * log((1-p0)/(1-p1)) / denominator

listofedit = []
labels_save,_ = CALC_NN(data, label)  
res = []
modilabel = label

for i in range(data.shape[0]):
    
    if n_class > 2:
        nlabel = change_label_multi(labels_save, i, A, R, M)
        
    else:
        nlabel = change_label(labels_save, i, A, R ,M)
    
    if nlabel != modilabel[i]:
        listofedit.append(i)
        modilabel[i] = nlabel
        
for i in range(testlabel.shape[0]):

    order=array(calc_NN(data, testdata[i]))
    labels=modilabel[order]
    res.append(Counter(labels[:ks]).most_common(1)[0][0]) 

    
print("acc:", sum(res == testlabel)/testlabel.shape[0])

************************************************************
alpha:0.05, beta:0.05, p0:0.05, p1:0.2, k:5
H0: p = 0.05  ************  H1: p = 0.2
Accuracy:0.8054114196998446
standard deviation:0.015345723118934894
The highest accuracy so far: 0.8054114196998446
************************************************************
alpha:0.05, beta:0.05, p0:0.05, p1:0.35000000000000003, k:5
H0: p = 0.05  ************  H1: p = 0.35000000000000003
Accuracy:0.8107038123167154
standard deviation:0.016855234569730232
The highest accuracy so far: 0.8107038123167154
************************************************************
alpha:0.05, beta:0.05, p0:0.05, p1:0.5000000000000001, k:5
H0: p = 0.05  ************  H1: p = 0.5000000000000001
Accuracy:0.804823184405727
standard deviation:0.013957033016764866
The highest accuracy so far: 0.8107038123167154
************************************************************
alpha:0.05, beta:0.05, p0:0.05, p1:0.6500000000000001, k:5
H0: p = 0.05  ************  H1: p = 

************************************************************
alpha:0.05, beta:0.25, p0:0.2, p1:0.5000000000000001, k:5
H0: p = 0.2  ************  H1: p = 0.5000000000000001
Accuracy:0.7918820079351387
standard deviation:0.020711752515760355
The highest accuracy so far: 0.8107038123167154
************************************************************
alpha:0.05, beta:0.25, p0:0.2, p1:0.6500000000000001, k:5
H0: p = 0.2  ************  H1: p = 0.6500000000000001
Accuracy:0.7901173020527859
standard deviation:0.019623966013010332
The highest accuracy so far: 0.8107038123167154
************************************************************
alpha:0.05, beta:0.25, p0:0.2, p1:0.8000000000000002, k:5
H0: p = 0.2  ************  H1: p = 0.8000000000000002
Accuracy:0.7901224771433499
standard deviation:0.017519205991711095
The highest accuracy so far: 0.8107038123167154
************************************************************
alpha:0.05, beta:0.25, p0:0.2, p1:0.9500000000000002, k:5
H0: p = 0.2  *

************************************************************
alpha:0.05, beta:0.45, p0:0.35000000000000003, p1:0.9500000000000002, k:5
H0: p = 0.35000000000000003  ************  H1: p = 0.9500000000000002
Accuracy:0.7907107124374676
standard deviation:0.013472897693228007
The highest accuracy so far: 0.8107038123167154
************************************************************
alpha:0.05, beta:0.45, p0:0.5000000000000001, p1:0.6500000000000001, k:5
H0: p = 0.5000000000000001  ************  H1: p = 0.6500000000000001
Accuracy:0.7877609108159392
standard deviation:0.02068171922264581
The highest accuracy so far: 0.8107038123167154
************************************************************
alpha:0.05, beta:0.45, p0:0.5000000000000001, p1:0.8000000000000002, k:5
H0: p = 0.5000000000000001  ************  H1: p = 0.8000000000000002
Accuracy:0.7936518889080558
standard deviation:0.015920517527058226
The highest accuracy so far: 0.8107038123167154
******************************************

************************************************************
alpha:0.05, beta:0.6500000000000001, p0:0.6500000000000001, p1:0.9500000000000002, k:5
H0: p = 0.6500000000000001  ************  H1: p = 0.9500000000000002
Accuracy:0.7907107124374676
standard deviation:0.013472897693228007
The highest accuracy so far: 0.8107038123167154
************************************************************
alpha:0.05, beta:0.6500000000000001, p0:0.8000000000000002, p1:0.9500000000000002, k:5
H0: p = 0.8000000000000002  ************  H1: p = 0.9500000000000002
Accuracy:0.7907107124374676
standard deviation:0.013472897693228007
The highest accuracy so far: 0.8107038123167154
************************************************************
alpha:0.05, beta:0.8500000000000001, p0:0.05, p1:0.2, k:5
H0: p = 0.05  ************  H1: p = 0.2
Accuracy:0.7907107124374676
standard deviation:0.013472897693228007
The highest accuracy so far: 0.8107038123167154
***********************************************************